In [1]:
import numpy as np
import pandas as pd
from scipy import linalg
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

np.set_printoptions(threshold=100)

In [2]:
# データの読み込み（MNIST手書き文字）
train_df = pd.read_csv('data/mnist-in-csv/mnist_train.csv', sep=',')
test_df = pd.read_csv('data/mnist-in-csv/mnist_test.csv', sep=',')

train_data = train_df.iloc[:,1:].to_numpy(dtype='float32')
train_target = train_df.iloc[:,0].to_numpy(dtype='int8')
train_target = OneHotEncoder(sparse=False).fit_transform(train_target.reshape(-1, 1))

test_data = test_df.iloc[:,1:].to_numpy(dtype='float32')
test_target = test_df.iloc[:,0].to_numpy(dtype='int8')
test_target = OneHotEncoder(sparse=False).fit_transform(test_target.reshape(-1, 1))

/home/mech-user/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/mech-user/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'"

In [3]:
# 重要パラメータの定義
# データの性質関連
TRAIN_SIZE = 20000       # 訓練データ数
TEST_SIZE = 2000         # テストデータ数
STEPS_PER_DATA = 10      # 1つのデータを何ステップ入れ続けるか
NUM_INPUT_NODES = 784    # 入力の次元数
NUM_OUTPUT_NODES = 10    # 出力の次元数

# リザバー関連
LEAK_RATE=0.4
SPECTRAL_RADIUS = 0.3
NUM_RESERVOIR_NODES = 1000 # リザバー内のユニット数
# BIAS = 1.0

In [4]:
# 画像をいっぺんに（784次元のまま）入力する場合
train_data = train_data[:TRAIN_SIZE]
test_data = test_data[:TEST_SIZE]
train_target = train_target[:TRAIN_SIZE]
test_target = test_target[:TEST_SIZE]

In [5]:
train_data.shape

(20000, 784)

In [6]:
test_data.shape

(2000, 784)

In [7]:
train_target.shape

(20000, 10)

In [8]:
test_target.shape

(2000, 10)

In [9]:
# 画像をいっぺんに（784次元のまま）入力する場合
train_data = np.array([i for i in train_data for _ in range(STEPS_PER_DATA)])
test_data = np.array([i for i in test_data for _ in range(STEPS_PER_DATA)])
train_target = np.array([i for i in train_target for _ in range(STEPS_PER_DATA)])

In [10]:
# 活性化関数の別例（利用しない）
def ReLU(x):
    return np.maximum(0, x)

# ある行の最大値を1、それ以外を0にする関数
def max_to_one(row):
    index = np.argmax(row)
    row = np.zeros(len(row))
    row[index] = 1
    return row

# ***この関数は不要になった***
# parentは2次元numpy配列で、childが足したい行（1次元numpy配列）
# np.append(parent, [child], axis=0)に等しい
# def numpy_append(parent, child):
#     parent_list = parent.tolist()
#     child_list = child.tolist()
#     parent_list.append(child_list)
#     parent = np.asarray(parent_list)
#     return parent

class ReservoirNetWork: # 入れるtrain_data, test_data, targetは全てnumpy配列

    def __init__(self, train_data, num_input_nodes, num_reservoir_nodes, num_output_nodes, leak_rate=0.1, activator=np.tanh):
        self.train_data = train_data
        self.log_reservoir_nodes = np.zeros((len(train_data), num_reservoir_nodes), dtype='float32') # reservoir層のノードの状態を記録

        # init weights
        self.weights_input = self._generate_input_weights(num_input_nodes, num_reservoir_nodes).astype('float32')
        self.weights_reservoir = self._generate_reservoir_weights(num_reservoir_nodes).astype('float32')
        self.weights_output = np.zeros((num_reservoir_nodes, num_output_nodes), dtype='float32')

        # それぞれの層のノードの数
        self.num_input_nodes = num_input_nodes
        self.num_reservoir_nodes = num_reservoir_nodes
        self.num_output_nodes = num_output_nodes

        self.leak_rate = leak_rate # 漏れ率
        self.activator = activator # 活性化関数

    # reservoir層のノードの次の状態を取得
    def _get_next_reservoir_nodes(self, input, current_state):
        next_state = (1 - self.leak_rate) * current_state
        next_state += self.leak_rate * (input @ self.weights_input + current_state @ self.weights_reservoir)
        return self.activator(next_state)

    # 出力層の重みを更新
    def _update_weights_output(self, target, lambda0):
        # Ridge Regression
        E_lambda0 = np.identity(self.num_reservoir_nodes) * lambda0
        inv_x = np.linalg.inv(self.log_reservoir_nodes.T @ self.log_reservoir_nodes + E_lambda0)
        # update weights of output layer
        self.weights_output = ((inv_x @ self.log_reservoir_nodes.T) @ target)

        
    # 学習する
    def train(self, lambda0=0.1):
        
        # 入力を1つずつ入れていく
        for i in range(len(self.train_data)):
            tr = self.train_data[i]
            current_state = self.log_reservoir_nodes[max(0,i-1)]
                
            self.log_reservoir_nodes[i] = self._get_next_reservoir_nodes(tr, current_state)   # 内部状態更新
            
            # 元々はappendを利用して書いていたが、遅いからやめた
            # self.log_reservoir_nodes = numpy_append(self.log_reservoir_nodes, self._get_next_reservoir_nodes(tr, current_state))
            # self.log_reservoir_nodes = np.append(self.log_reservoir_nodes, [self._get_next_reservoir_nodes(tr, current_state)], axis=0)
            
            if i % 1000 == 0:
                print('training data no. {}\n'.format(i))
                
        # まとめて行列計算で重みを更新
        self._update_weights_output(train_target, lambda0)
        
        
    # 予測する
    def predict(self, test_data):
        reservoir_nodes = self.log_reservoir_nodes[-1] # 訓練の結果得た最後の内部状態を取得
        
        # 空の配列を用意
        predict_output = np.zeros((len(test_data), (NUM_OUTPUT_NODES)), dtype='float32')
        reduced_predict_output = np.zeros(( int(len(test_data)/STEPS_PER_DATA), NUM_OUTPUT_NODES), dtype='float32')
        tmp_array = np.zeros(NUM_OUTPUT_NODES)
        tmp_count = 0
        
        # 入力を1つずつ入れていく
        for i in range(len(test_data)):
            te = test_data[i]
            reservoir_nodes = self._get_next_reservoir_nodes(te, reservoir_nodes) # 内部状態更新
            predict_output[i] = max_to_one( self.get_output(reservoir_nodes) ) # 内部状態を読み出して出力を得る、出力が最大のところを1とする
            # predict_output = numpy_append(predict_output, output)
            # predict_output = np.append(predict_output, [output], axis=0)
            # predict_output = predict_output[1:]
            
            tmp_array += predict_output[i]
            
            # 画像1枚分の入力が終わるごとに、出力結果を記録。tmp_countは「何枚目の画像」かを示す
            if(i%STEPS_PER_DATA == STEPS_PER_DATA - 1):
                reduced_predict_output[tmp_count] = tmp_array
                # reduced_predict_output = numpy_append(reduced_predict_output, tmp_array)
                # reduced_predict_output = np.append(reduced_predict_output, [tmp_array], axis=0)
                tmp_array = np.zeros(NUM_OUTPUT_NODES)
                tmp_count += 1    
                
        # 各画像について、多数決で出力を決める
        final_predict_output = [max_to_one(row) for row in reduced_predict_output]
        
        return final_predict_output, predict_output, reduced_predict_output

    
    # 内部状態から出力を計算
    def get_output(self, reservoir_nodes):
        return reservoir_nodes @ self.weights_output 

    #############################
    ##### private method ########
    #############################

    # 入力層の重み：0.1か0か-0.1で初期化したものを返す
    def _generate_input_weights(self, num_input_nodes, num_reservoir_nodes):
        return np.random.choice([-0.1, 0, 0.1], num_input_nodes*num_reservoir_nodes, p=[0.1, 0.8, 0.1]).reshape(num_input_nodes, num_reservoir_nodes)

    # Reservoir層の重みを初期化
    def _generate_reservoir_weights(self, num_nodes):
        # 0以上1以下のランダムな値
        weights = np.random.normal(0, 1, num_nodes * num_nodes)
        
        # ランダムに90%を選び、0とする（疎な行列にする）
        indices = np.random.choice(np.arange(len(weights)), replace=False, size=int(len(weights) * 0.9))
        weights[indices] = 0
        
        weights = weights.reshape([num_nodes, num_nodes])
        spectral_radius = max(abs(linalg.eigvals(weights)))
        return weights / spectral_radius * SPECTRAL_RADIUS


In [11]:
# モデルの定義
model = ReservoirNetWork(train_data=train_data,
    num_input_nodes=NUM_INPUT_NODES,
    num_reservoir_nodes=NUM_RESERVOIR_NODES,
    num_output_nodes=NUM_OUTPUT_NODES,
    leak_rate=LEAK_RATE)

model.train() # 訓練

# 予測結果
final_predict_output, predict_output, reduced_predict_output = model.predict(test_data)


training data no. 0

training data no. 1000

training data no. 2000

training data no. 3000

training data no. 4000

training data no. 5000

training data no. 6000

training data no. 7000

training data no. 8000

training data no. 9000

training data no. 10000

training data no. 11000

training data no. 12000

training data no. 13000

training data no. 14000

training data no. 15000

training data no. 16000

training data no. 17000

training data no. 18000

training data no. 19000

training data no. 20000

training data no. 21000

training data no. 22000

training data no. 23000

training data no. 24000

training data no. 25000

training data no. 26000

training data no. 27000

training data no. 28000

training data no. 29000

training data no. 30000

training data no. 31000

training data no. 32000

training data no. 33000

training data no. 34000

training data no. 35000

training data no. 36000

training data no. 37000

training data no. 38000

training data no. 39000

training data

In [12]:
# 誤差
np.sum(np.absolute(final_predict_output - test_target)) * 0.5 / len(test_target)

0.1165

In [14]:
# 正解のデータ
len(test_target.tolist())

2000

In [15]:
# 予測値（画像ごと）
len(final_predict_output)

2000

In [16]:
# 各ステップごとの予測値
len(predict_output.tolist())

20000

In [17]:
# 画像ごとの多数決の結果
len(reduced_predict_output.tolist())

2000

In [18]:
# 以下、比較対象とするために線形回帰とRidge回帰も試してみる
# データ読み込み
train_df = pd.read_csv('data/mnist-in-csv/mnist_train.csv', sep=',')
test_df = pd.read_csv('data/mnist-in-csv/mnist_test.csv', sep=',')

train_data = train_df.iloc[:,1:].to_numpy(dtype='float32')
train_target = train_df.iloc[:,0].to_numpy(dtype='int8')
train_target = OneHotEncoder(sparse=False).fit_transform(train_target.reshape(-1, 1))

test_data = test_df.iloc[:,1:].to_numpy(dtype='float32')
test_target = test_df.iloc[:,0].to_numpy(dtype='int8')
test_target = OneHotEncoder(sparse=False).fit_transform(test_target.reshape(-1, 1))

/home/mech-user/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/mech-user/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'"

In [19]:
train_data = train_data[:TRAIN_SIZE]
test_data = test_data[:TEST_SIZE]
train_target = train_target[:TRAIN_SIZE]
test_target = test_target[:TEST_SIZE]

In [20]:
from sklearn.linear_model import LinearRegression,Ridge

lr = LinearRegression()
ridge = Ridge(alpha=0.1)

lr.fit(train_data, train_target)
ridge.fit(train_data, train_target)

/home/mech-user/.local/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=3.07117e-12): result may not be accurate.
  overwrite_a=True).T


Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [21]:
pred_lr = lr.predict(test_data)
pred_ridge = ridge.predict(test_data)

In [22]:
for i in range(len(pred_lr)):
    pred_lr[i] = max_to_one(pred_lr[i])
    pred_ridge[i] = max_to_one(pred_ridge[i])

In [23]:
np.sum(np.absolute(pred_lr - test_target)) * 0.5 / len(test_target)

0.1895

In [24]:
np.sum(np.absolute(pred_ridge - test_target)) * 0.5 / len(test_target)

0.1905